In [477]:
## give google sheet access to google colab
from google.colab import auth
import gspread
from google.auth import default
#autenticating to google
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [478]:
# read sheet

import pandas as pd
#defining my worksheet
worksheet = gc.open('Retro-TestGSheetToCode').sheet1
#get_all_values gives a list of rows
rows = worksheet.get_all_values()
#Convert to a DataFrame 
df = pd.DataFrame(rows)

In [479]:
#creating columns name
df.columns = df.iloc[6]
df = df.iloc[7:]

df.head(5)

script = df

## 场景def

In [480]:

def scene_settings(scene_name): #背景图片
  if scene_name != '':
    cmd_lst = []
    scene_cmd = scene_name.split(',') # got initial showing scene not transition
    if len(scene_cmd) == 1: # transition scene, need to also make character disappear
      set_scene_name = '$s_current = ' + "'" + scene_name + "'"
      call_func = 'call trans_scene_1'
      cmd_lst.append(set_scene_name)
      cmd_lst.append(call_func)
    else: #initial showing
      set_scene_name = '$scene_name = ' + "'" + scene_cmd[0] + "'"
      call_func = 'call show_scene_1'
      cmd_lst.append(set_scene_name)
      cmd_lst.append(call_func)
    # cmd_lst.append("\n")
    return cmd_lst
  else:
    return []


def CG_settings(CG_name): # CG
  if CG_name != '':
    cg_cmd = CG_name.split(" ")
    act = cg_cmd[0]
    name = cg_cmd[1]
    if act == 'show':
      a1 = '$ cg_name = "' + name + '"'
      a2 = 'call show_CG'
      return [a1, a2]
    else:
      a2 = 'call hide_CG'
      a3 = 'call show_scene_1'
      return [a2, a3]
  else:
    return []



##人物def

In [481]:

def sprite_position(sprites): #人物位置
  if sprites != '':
    stand_cmd = []
    sprite_lst = sprites.split(", ")
    for s in sprite_lst:
      s_detail = s.split(" ")
      act = s_detail[0]
      name = s_detail[1]
      pos = s_detail[2]
      cmd = act + ' ' + name + ' onlayer inyourface at ' + pos
      stand_cmd.append(cmd)
    # stand_cmd.append("\n")
    return stand_cmd
  else:
    return []
    

def expression_set(exp_name): # 表情设置
  if exp_name != '':
    character_lst = exp_name.split(", ")
    exp_lst = []
    for i in character_lst:
      a = i.split(' ')
      name = a[0]
      exp = a[1]
      ans = '$ ' + name + '_expressions = "' + exp + '"'
      exp_lst.append(ans)
    return exp_lst
  else:
    return []
    

def sprite_face(face_element):
  if face_element != '':
    s_lst = []
    s = face_element.split(",")
    for i in s:
      n = i.split(' ')
      name = n[0]
      boo = n[1]
      ans = '$ ' + name + ' = ' + boo
      s_lst.append(ans)
    return s_lst
  else:
    return []


## 其他def

In [482]:
def label_creation(label_name):
  if label_name != '':
    return 'label '+label_name+':'
  else:
    return ''

def set_variables(var_set_command): #变量
  if var_set_command != '':
    command_list = var_set_command.split(", ")
    ans_lst = []
    for var in command_list:
      ans = '$ ' + var
      ans_lst.append(ans)
    # ans_lst.append("\n")
    return ans_lst
  else:
    return []


def jump_label(label_name): # 跳跃分支
  if label_name != '':
    return 'call ' + label_name


## 音乐
def music_label(music_name, volume):
  if music_name != '':
    if music_name != 'STOP':
      n1 = 'play music "' + music_name
      n2 = ''
      if volume != '':
        n2 = '" volume ' + volume
      return n1+n2
    else:
      return 'stop music fadeout 1.0'
  else:
    return ''

## Audio

def audio_label(audio_name):
  if audio_name != '':
    return 'play sound "' + audio_name + '"'
  else:
    return ''

def music_label(music_name, volume):
  if music_name != '':
    if music_name != 'STOP':
      n1 = 'play music "' + music_name
      n2 = ''
      if volume != '':
        n2 = '" volume ' + volume
      return n1+n2
    else:
      return 'stop music fadeout 1.0'
  else:
    return ''


## Variables And Conditions etc

In [483]:
def condition_boo (condition_statement):
  if condition_statement != '':
    if condition_statement == 'in':
      return 'in-line'
    else:
      ans = 'if ' + condition_statement + ":"
      return ans
  else:
    return ''



def talking_dialogue(dialogue_type, character, dialogue, label_name, condition_statement):
  d_lst = []
  var_code = set_variables(row['Set Variables']) # set variables
  if character != '':
    character = character + ' '

  if dialogue != '':
    if dialogue_type == 'Menu':
      d_lst.append("menu:")
      d_lst.append("     " + character + '"' + dialogue + '"')#对话
      if len(var_code) >0:
        for i in var_code:
          d_lst.append("     " + i)

  #### 菜单选项
    elif dialogue_type == 'Selection':
      d_lst.append("     " + character + '"' + dialogue + '"' + ':')#对话
      if len(var_code) >0: #变量
        for i in var_code:
          d_lst.append("          " + i)

  #### 菜单选项相应对白
    elif dialogue_type == 'Inline':

      if len(var_code) >0: #变量
        for i in var_code:
          d_lst.append("          " + i)

      d_lst.append("          " + character + '"' + dialogue + '"')#对话
      if label_name != '': #跳分支
        d_lst.append("          " + 'call ' + label_name)

  #### 普通对白
    else:
      condition = condition_boo (condition_statement)
      
      # 如果没有condition
      if condition == '': 
        if len(var_code) >0:
            for i in var_code:
              d_lst.append(i)
        ans = character + '"' + dialogue + '"'#对话
        d_lst.append(ans)
        if label_name != '': #跳分支
          d_lst.append('call ' + label_name)

      #condition之下相关语句
      elif condition == 'in-line': 
        if len(var_code) >0:
            for i in var_code:
              d_lst.append("     " + i)
        ans = "     " + character + '"' + dialogue + '"'#对话
        d_lst.append(ans)
        if label_name != '': #跳分支
          d_lst.append("     " + 'call ' + label_name)


      # condition if + 语句
      else:
          d_lst.append(condition)
          print(condition)
          if len(var_code) >0:
            for i in var_code:
              d_lst.append("     " + i)
          ans = "     " + character + '"' + dialogue + '"'#对话
          d_lst.append(ans)
          if label_name != '': #跳分支
            d_lst.append("     " + 'call ' + label_name)
        
    return d_lst

  else:
    return []


## Run

In [484]:
# 创建单个rpy file

f= open("test.rpy","w+")

In [485]:
# loop每一行，写code

for idx, row in script.iterrows(): # 一行一行

  label_code = label_creation(row['Label Name']) # create label
  if label_code != '':
    f.write(label_code)
    f.write("\n")

### 音效等

  audio_code = audio_label(row['Sound Effect'])
  if audio_code != '':
    f.write("     ")
    f.write(audio_code)
    f.write("\n")

  music_code = music_label(row['Music'], row['Volume'])
  if music_code != '':
    f.write("     ")
    f.write(music_code)
    f.write("\n")

  
### 背景
  scene_code = scene_settings(row['Scene']) # set scene
  if len(scene_code) >0:
    for i in scene_code:
      f.write("     ")
      f.write(i)
      f.write("\n")
      
### 角色sprite
  sprite_move_cmd = sprite_position(row['Sprites']) # set sprite position
  if len(sprite_move_cmd) > 0:
    for i in sprite_move_cmd:
      f.write("     ")
      f.write(i)
      f.write("\n")

### CG
  CG_code = CG_settings(row['CG'])
  if len(CG_code) > 0:
    for i in CG_code:
      f.write("     ")
      f.write(i)
      f.write("\n")


### 角色表情
  exp_cmd = expression_set(row['Expression'])
  if len(exp_cmd) > 0:
    for i in exp_cmd:
      f.write("     ")
      f.write(i)
      f.write("\n")

  face_addon = sprite_face(row['Face Elements'])
  if len(face_addon) > 0:
    for i in face_addon:
      f.write("     ")
      f.write(i)
      f.write("\n")


### 跳跃分支
  dialogue = talking_dialogue(row['Dialogue_type'], row['Character'], row['Dialogue'], row['Jump Label'], row['Conditions'])
  if len(dialogue) > 0:
    for i in dialogue:
      f.write("     ")
      f.write(i)
      f.write("\n")
    
  f.write("\n")
  
  

f.close()

if choice == 1:
if choice == 2:


In [486]:
#read rpy file
f=open("test.rpy", "r")
if f.mode == 'r':
  contents =f.read()

contents

'label act1_main1:\n     $scene_name = \'rich_mansion_outside\'\n     call show_scene_1\n     show hendrix onlayer inyourface at middle_r\n     show edmund onlayer inyourface at middle_l\n     $ hendrix_expressions = "sad"\n     $ hendrix_moving_eyes = True\n     $ hendrix_ver=\'hendrix_a\'\n     $ edmund_ver=\'edmund_a\'\n     Hendrix "测试开始了。"\n\n     $ hendrix_moving_eyes = False\n     Hendrix "接下来应该播放华尔兹,并且更换背景了。"\n\n     play music "M&S/test.mp3" volume 0.1\n     $s_current = \'campusUni\'\n     call trans_scene_1\n     "播放音乐中。"\n\n     Hendrix "接下来是我的cg！"\n\n     hide hendrix onlayer inyourface at middle_r\n     hide edmund onlayer inyourface at middle_l\n     $ cg_name = "Hendrix/meet"\n     call show_CG\n     "cg displaying"\n\n     show hendrix onlayer inyourface at middle_r\n     show edmund onlayer inyourface at middle_l\n     call hide_CG\n     call show_scene_1\n     $ edmund_expressions = "serious"\n     $ edmund_dark = True\n     Edmund "为什么只有他有cg！"\n\n     $ hendrix_expr

In [487]:
# 下载renpy file
from google.colab import files
files.download("test.rpy") 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>